# 02 - MCP Tools

Demonstrate various usage scenarios for MCP tools with the AIMU MCPClient class.

## A - MCP Client with File Server

The simplest way to create an MCPClient is by pointing it to a Python file that implements an MCP server. This is useful for testing and development.

In [ ]:
from aimu.tools import MCPClient
from aimu import paths

# Required to allow nested event loops in Jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

client = MCPClient(file=str(paths.package / "tools" / "servers.py"))
client.list_tools()

Once connected, you can list all available tools provided by the MCP server. This shows the tools, their descriptions, and required parameters.

In [ ]:
client.call_tool("echo", {"echo_string": "Hello, world!"})

You can call individual tools by their name and pass the required parameters. The tool will execute and return its result.

## B - MCP Client with Configuration

For more complex setups, you can provide a configuration dictionary that defines multiple MCP servers. This allows you to connect to multiple servers simultaneously.

In [ ]:
config = {
    "mcpServers": {"echo": {"command": "python", "args": [str(paths.package / "tools" / "servers.py")]}},
}

client = MCPClient(config=config)
client.list_tools()

The configuration-based client provides the same interface for listing and calling tools, but can aggregate tools from multiple servers.

In [ ]:
client.call_tool("echo", {"echo_string": "Hello, world!"})

Tools work exactly the same way regardless of how the MCPClient was initialized.

## C - MCP Client with FastMCP Server

You can also create an MCPClient directly from a FastMCP server instance. This is particularly useful when you want to define tools inline or when integrating with existing FastMCP applications.

In [ ]:
from fastmcp import FastMCP

mcp = FastMCP("AIMU Tools")


@mcp.tool()
def echo(echo_string: str) -> str:
    return echo_string


client = MCPClient(server=mcp)
client.list_tools()

This approach creates an MCP server with tools defined using Python decorators, then wraps it with an MCPClient for easy access.

In [ ]:
client.call_tool("echo", {"echo_string": "Hello, world!"})

The FastMCP-based client works identically to file-based and configuration-based clients, providing a consistent interface for tool interaction.

## D - Multiple parallel tool calls

Show how LLMs can use mutiple tools in parallel by making concurrent calls to different MCP tools and aggregating the results.

In [ ]:
from aimu.tools import MCPClient
from fastmcp import FastMCP

# Required to allow nested event loops in Jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

mcp = FastMCP("AIMU Tools")


@mcp.tool()
def get_current_temperature(location: str) -> str:
    """
    Get the current temperature for a given location.

    Args:
        location (str): The location to get the temperature for.

    Returns:
        str: The current temperature in Celsius.
    """

    return "21.0"


@mcp.tool()
def get_current_weather_conditions(location: str) -> str:
    """
    Get the current weather conditions for a given location.

    Args:
        location (str): The location to get the weather conditions for.

    Returns:
        str: The current weather conditions.
    """

    return "sunny"


mcp_client = MCPClient(server=mcp)
mcp_client.list_tools()

In [ ]:
from aimu.models import OllamaClient as ModelClient

model_client = ModelClient(ModelClient.MODELS.QWEN_3_8B)
model_client.mcp_client = mcp_client

In [ ]:
model_client.chat("What is the current temperature and weather conditions in New York City?")

In [ ]:
model_client.messages